In [1]:
import os
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np
import cv2

In [2]:
def generate_lp_number():
    allowed_letters = 'BDFGHJKLMNOPRSTVWXYZ'
    allowed_numbers = '0123456789'

    cases = [
        # Case 1: 99-XX-XX
        random.choice(allowed_numbers) + random.choice(allowed_numbers) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters),

        # Case 2: 99-XXX-9
        random.choice(allowed_numbers) + random.choice(allowed_numbers) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters) + random.choice(allowed_letters) + '-' + random.choice(allowed_numbers),

        # Case 3: 9-XXX-99
        random.choice(allowed_numbers) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters) + random.choice(allowed_letters) + '-' + random.choice(allowed_numbers) + random.choice(allowed_numbers),

        # Case 4: XX-999-X
        random.choice(allowed_letters) + random.choice(allowed_letters) + '-' + random.choice(allowed_numbers) + random.choice(allowed_numbers) + random.choice(allowed_numbers) + '-' + random.choice(allowed_letters),

        # Case 5: X-999-XX
        random.choice(allowed_letters) + '-' + random.choice(allowed_numbers) + random.choice(allowed_numbers) + random.choice(allowed_numbers) + '-' + random.choice(allowed_letters) + random.choice(allowed_letters)
    ]
    return random.choice(cases)


def generate_lp_image():
    font = ImageFont.truetype('../lp_template_files/font_dutch_lp.ttf', 220)
    lp_image = Image.open('../lp_template_files/template_dutch_lp.png')

    draw = ImageDraw.Draw(lp_image)

    license_plate = generate_lp_number()

    # Calculate the width and height of the text
    text_bbox = draw.textbbox((0, 0), license_plate, font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # Calculate the position of the text
    text_x = (lp_image.width - text_width) / 2 + 60
    text_y = (lp_image.height - text_height) / 2 - 45

    draw.text((text_x, text_y), license_plate, fill='black', font=font)

    return lp_image

In [14]:
def get_Rx(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def get_Ry(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def get_Rz(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def rotate_2d_point_in_3d(point, theta_x, theta_y, theta_z):
    point = np.array([point[0], point[1], 0])
    point = np.dot(get_Rx(theta_x), point)
    point = np.dot(get_Ry(theta_y), point)
    point = np.dot(get_Rz(theta_z), point)
    return point[:2]

def translate_points_to_image(points, image_size):
    point1, point2, point3, point4 = points

    # Calculate the minimum and maximum x and y values of the points
    min_x = min(point1[0], point2[0], point3[0], point4[0])
    min_y = min(point1[1], point2[1], point3[1], point4[1])
    max_x = max(point1[0], point2[0], point3[0], point4[0])
    max_y = max(point1[1], point2[1], point3[1], point4[1])

    # Calculate the translation values
    dx = max(0, -min_x, max_x - image_size)
    dy = max(0, -min_y, max_y - image_size)

    translated_points = [(point[0] + 1.1 * dx, point[1] + 1.1 * dy) for point in points]

    return translated_points

In [15]:
def get_bounding_box_points(img_size):
    scale = np.random.uniform(2, 6)
    lp_dimensions = (52 * scale, 11 * scale) # Dutch license plates are 52x11 cm
    x = np.random.uniform(0, img_size - lp_dimensions[0])
    y = np.random.uniform(0, img_size - lp_dimensions[1])

    point1 = (x, y)
    point2 = (x + lp_dimensions[0], y)
    point3 = (x + lp_dimensions[0], y + lp_dimensions[1])
    point4 = (x, y + lp_dimensions[1])

    return [point1, point2, point3, point4]

def random_rotation(points):
    theta_x = np.deg2rad(random.uniform(-60, 60))
    theta_y = np.deg2rad(random.uniform(-60, 60))
    theta_z = np.deg2rad(random.uniform(-10, 10))

    point1 = rotate_2d_point_in_3d(points[0], theta_x, theta_y, theta_z)
    point2 = rotate_2d_point_in_3d(points[1], theta_x, theta_y, theta_z)
    point3 = rotate_2d_point_in_3d(points[2], theta_x, theta_y, theta_z)
    point4 = rotate_2d_point_in_3d(points[3], theta_x, theta_y, theta_z)

    return [point1, point2, point3, point4]

In [56]:
def create_img_with_lp(background_img, img_size):
    background_img.resize((img_size, img_size))
    lp_img = generate_lp_image()

    points = get_bounding_box_points(img_size)
    points = random_rotation(points)
    points = translate_points_to_image(points, img_size)

    box_points_lp = [(0, 0), (lp_img.width, 0), (lp_img.width, lp_img.height), (0, lp_img.height)]

    matrix = cv2.getPerspectiveTransform(np.float32(box_points_lp), np.float32(points))

    transformed_lp_img = cv2.warpPerspective(np.array(lp_img), matrix, (img_size, img_size))

    transformed_lp_img = Image.fromarray(transformed_lp_img)

    background_img.paste(transformed_lp_img, (0, 0), transformed_lp_img)

    return background_img, points

In [61]:
def create_dataset(raw_image_path, amount, img_size):
    image_files = os.listdir(raw_image_path)
    
    for i in range(amount):

        # Load the background image
        background_image = Image.open(os.path.join(raw_image_path, image_files[i])).resize((img_size, img_size))
        
        # Create the image with the license plate and the points of the license plate bounding box
        img_with_lp, points = create_img_with_lp(background_image, img_size)
        
        # Save the image and the label
        img_with_lp.save(f'../../images/synthetic_data/images/{i}.png')
        with open(f'../../images/synthetic_data/labels/{i}.txt', 'w') as f:
            f.write(f'0 {points[0][0]/img_size} {points[0][1]/img_size} {points[1][0]/img_size} {points[1][1]/img_size} {points[2][0]/img_size} {points[2][1]/img_size} {points[3][0]/img_size} {points[3][1]/img_size}')

        
create_dataset('../../images/raw_random', 1000, 640)


OSError: image file is truncated (89 bytes not processed)